# Evaluate Random Forest Model

In [132]:
import pandas as pd
import pickle
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS

import xgboost as xgb
from sklearn import metrics

## 1. Load the training data

In [339]:
flight_sample = pd.read_csv('flight_sample1.zip', compression='zip')
flight_sample.head()


,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,early_count,early_avgDryDays,early_avgRainDays,early_avgCloud,early_avgSnowDays,delay_count,delay_avgDryDays,delay_avgRainDays,delay_avgCloud,delay_avgSnowDays
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,55.5,17,11,3,0,30.8,18,10,3,0
1,DL,DL_CODESHARE,4016,OO,N292SY,4016,12953,13342,1747,1929.0,...,55.5,17,11,3,0,30.8,18,10,3,0
2,DL,DL_CODESHARE,4016,OO,N278SY,4016,12953,13342,1747,1750.0,...,55.5,17,11,3,0,30.8,18,10,3,0
3,DL,DL_CODESHARE,3616,OO,N268SY,3616,12953,11995,1729,1851.0,...,55.5,17,11,3,0,30.8,18,10,3,0
4,DL,DL_CODESHARE,3607,OO,N276SY,3607,12953,11995,910,906.0,...,55.5,17,11,3,0,30.8,18,10,3,0


In [340]:
flight_sample = flight_sample.sample(n=30000, random_state=1)

### Drop all delay columns

In [341]:
pruned_data = flight_sample.drop(columns=[
    'mkt_unique_carrier','branded_code_share','mkt_carrier_fl_num','op_unique_carrier','tail_num','op_carrier_fl_num','op_unique_carrier','tail_num','op_carrier_fl_num','origin_airport_id','dest_airport_id',
    'cancellation_code','dup','fl_date','state','year',
    'first_dep_time', 'dep_time', 'air_time', 'is_delayed', 
    'actual_elapsed_time', 'taxi_in', 'carrier_delay', 'cancelled', 'diverted',
    'total_add_gtime', 'longest_add_gtime',
    'arr_time','nas_delay', 'wheels_on', 'wheels_off','weather_delay', 
    'security_delay', 'dep_delay','late_aircraft_delay', 'taxi_out'])
pruned_data.columns

Index(['crs_dep_time', 'crs_arr_time', 'arr_delay', 'crs_elapsed_time',
       'flights', 'distance', 'flight_speed', 'flight_haul_type',
       'flight_month', 'arrival_hour_of_day', 'departure_hour_of_day',
       'percent_delay', 'percent_flights', 'avg_flights', 'avg_passengers',
       'total_distance', 'total_passengers', 'avg_fuel', 'avgCloud',
       'avgDryDays', 'avgRainDays', 'avgSnowDays', 'avgThunderDays',
       'early_count', 'early_avgDryDays', 'early_avgRainDays',
       'early_avgCloud', 'early_avgSnowDays', 'delay_count',
       'delay_avgDryDays', 'delay_avgRainDays', 'delay_avgCloud',
       'delay_avgSnowDays'],
      dtype='object')

## Remove NaNs

In [342]:
# Get columns with NaN values present
NaN_cols = pruned_data.columns[pruned_data.isnull().any()]

# Adjust NaN values by replacing them with respective column means to prevent data loss
for column in NaN_cols:
    pruned_data[column].fillna(value=pruned_data[column].mean(), inplace=True)

In [343]:
pruned_data[NaN_cols].isna().sum()

arr_delay    0
dtype: int64

In [344]:
pruned_data.head()

,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,flights,distance,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,...,early_count,early_avgDryDays,early_avgRainDays,early_avgCloud,early_avgSnowDays,delay_count,delay_avgDryDays,delay_avgRainDays,delay_avgCloud,delay_avgSnowDays
864377,1405,1530,-19.0,145.0,1.0,853.0,352.965517,0,3,15,...,56.8,19,10,4,2,28.5,19,9,4,2
554769,1125,1511,-13.0,166.0,1.0,942.0,340.481928,0,4,15,...,56.1,18,10,4,1,26.5,19,9,3,1
956373,705,848,255.0,103.0,1.0,408.0,237.669903,0,12,8,...,51.7,20,8,4,3,31.8,21,8,4,2
335821,2118,2301,9.0,103.0,1.0,481.0,280.194175,0,6,23,...,50.5,17,10,2,0,35.4,18,9,2,0
430891,2215,600,-34.0,285.0,1.0,2288.0,481.684211,1,1,6,...,52.4,21,7,4,4,26.1,21,7,4,3


In [345]:
y = pruned_data['arr_delay']

In [346]:
pruned_data = pruned_data.drop(columns=['arr_delay'])
X = pruned_data
X.shape

(30000, 32)

In [347]:
pruned_data.describe()

,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,...,early_count,early_avgDryDays,early_avgRainDays,early_avgCloud,early_avgSnowDays,delay_count,delay_avgDryDays,delay_avgRainDays,delay_avgCloud,delay_avgSnowDays
count,30000.000000,30000.000000,30000.000000,30000.0,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,1331.837967,1487.787033,140.678900,1.0,791.318667,298.208217,0.220467,6.581033,14.584500,13.045733,...,54.787730,18.717933,9.159367,3.051267,1.179300,28.825570,19.312867,8.555200,2.969633,0.946267
std,490.891986,517.751711,72.238174,0.0,592.760160,94.695521,0.450186,3.393310,5.174806,4.891377,...,4.054337,1.595992,1.795850,0.862204,1.501786,3.650017,1.656383,1.445184,0.814535,1.133706
min,4.000000,1.000000,20.000000,1.0,31.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,43.700000,17.000000,6.000000,2.000000,0.000000,22.100000,17.000000,6.000000,2.000000,0.000000
25%,916.000000,1104.000000,90.000000,1.0,362.000000,240.000000,0.000000,4.000000,11.000000,9.000000,...,52.400000,17.000000,8.000000,2.000000,0.000000,26.500000,18.000000,7.000000,2.000000,0.000000
50%,1325.000000,1515.000000,122.000000,1.0,629.000000,309.000000,0.000000,7.000000,15.000000,13.000000,...,56.800000,18.000000,10.000000,3.000000,0.000000,28.500000,19.000000,9.000000,3.000000,0.000000
75%,1735.000000,1920.000000,170.000000,1.0,1020.000000,365.947063,0.000000,10.000000,19.000000,17.000000,...,56.900000,20.000000,11.000000,4.000000,2.000000,31.800000,21.000000,10.000000,4.000000,2.000000
max,2359.000000,2359.000000,675.000000,1.0,5095.000000,543.000000,2.000000,12.000000,23.000000,23.000000,...,59.500000,21.000000,12.000000,4.000000,4.000000,35.400000,22.000000,11.000000,4.000000,3.000000


In [348]:
y.describe()

count    30000.000000
mean         5.510859
std         51.143783
min        -62.000000
25%        -15.000000
50%         -6.000000
75%          7.000000
max       1495.000000
Name: arr_delay, dtype: float64

## Set y to log(y)

In [349]:
print("Y Delay Skew", y.skew())
log_y = np.log(y+63)
print("Y Log Delay Skew", log_y.skew())

Arr Delay Skew 9.114441853067026
Arr Delay Skew 1.3774418111043554


In [350]:
y.shape

(30000,)

## Method to convert predicted values back from log e

In [435]:
def convert_pred_y(y_pred):
    return np.exp(y_pred,) - 63

In [436]:
sample_arr = np.asarray([2, 8, 32])
print(sample_arr)
log_sample_arr = np.log(sample_arr + 63)
print(log_sample_arr)
print(convert_pred_y(log_sample_arr))    

[ 2  8 32]
[4.17438727 4.26267988 4.55387689]
[ 2.  8. 32.]


## Convert to Arrays

In [351]:
X = X.to_numpy()

In [352]:
y = y.to_numpy()

In [353]:
print("The mean of all the target values is: ",y.mean())

The mean of all the target values is:  5.510858851918567


In [354]:
print("The mean of all the log target values is: ",log_y.mean())

The mean of all the log target values is:  4.112675573390643


In [355]:
ar_nan = np.where(np.isnan(log_y))
print (len(ar_nan[0]))
ar_inf = np.where(np.isinf(log_y))
print (ar_inf)

0
(array([], dtype=int64),)


### Split the data

In [356]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, log_y, test_size=0.25,random_state=109) # 75% training and 30% test

### Scale the data

In [357]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [358]:
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.svm import SVC

def runGridSearch(model, param_grid, X_train, y_train, X_test, y_test):
    grid = GridSearchCV(estimator=model, 
        param_grid=param_grid, 
        scoring='r2', 
        verbose=1, 
        n_jobs=-1) 
    
    best_model = grid.fit(X_train, y_train)


    y_pred = best_model.predict(X_test)
    
    print("MSE: ", metrics.mean_squared_error(y_test, y_pred))
    print("MAE: ", metrics.mean_absolute_error(y_test, y_pred))
    print("Variance Score: ", metrics.explained_variance_score(y_test, y_pred))
    print("R2: ", metrics.r2_score(y_test, y_pred))
    
    
    results = {}
    results['Name'] = model.__class__.__name__
    results['MSE'] = metrics.mean_squared_error(y_test, y_pred)
    results['MAE'] = metrics.mean_absolute_error(y_test, y_pred)
    results['Variance Score'] = metrics.explained_variance_score(y_test, y_pred)
    results['R2'] = metrics.r2_score(y_test, y_pred)
    grid_results_df = pd.DataFrame(results, index=[1])
    
    return grid_results_df, best_model
    

## Build Base Models Hyper Parameters

In [359]:
# RandomForestRegressor
param_grid_rf = {
    'n_estimators': [5,10, 100, 500],
    'max_depth': [5,10,100]
}

# DecisionTree
param_grid_dt = {    
    'max_depth': [5,10,100]
}

# XGBoost
param_grid_xgb = {
    'objective':['reg:squarederror'], 
    'colsample_bytree':[0.3], 
    'learning_rate':[0.1,0.01],
    'max_depth':[5,10,100], 
    'alpha':[1,0.1,0.01], 
    'n_estimators':[5,10,100,500]
    
}
param_grid_lr = {
   'fit_intercept':[True,False]
}

grid_results_df = pd.DataFrame(columns=['Name', 'MSE', 'MAE', 'Variance Score', 'R2'])

In [360]:
# Decision Tree
gridResult, dt_best_model = runGridSearch(DecisionTreeRegressor(), param_grid_dt, X_train, y_train, X_test, y_test)
grid_results_df = grid_results_df.append(gridResult, ignore_index=True)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    3.4s finished


MSE:  0.17596361018879483
MAE:  0.29168596033085425
Variance Score:  0.02114019222145902
R2:  0.021140188550326844


In [361]:
# Random Forest
gridResult, rf_best_model = runGridSearch(RandomForestRegressor(), param_grid_rf, X_train, y_train, X_test, y_test)
grid_results_df = grid_results_df.append(gridResult, ignore_index=True)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   54.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.9min finished


MSE:  0.173300228597076
MAE:  0.29014421400212864
Variance Score:  0.035956202275868376
R2:  0.035956190562852264


In [362]:
# XGBoost
gridResult, xg_best_model = runGridSearch(xgb.XGBRegressor(), param_grid_xgb, X_train, y_train, X_test, y_test)
grid_results_df = grid_results_df.append(gridResult, ignore_index=True)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 12.2min finished


MSE:  0.17315274745116865
MAE:  0.28529843585756504
Variance Score:  0.040201280678827556
R2:  0.036776606593873495


In [326]:
# Linear Regression
gridResult, lr_best_model = runGridSearch(LinearRegression(), param_grid_lr, X_train, y_train, X_test, y_test)
grid_results_df = grid_results_df.append(gridResult, ignore_index=True)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished


MSE:  0.1736904365646565
MAE:  0.2908172624375203
Variance Score:  0.033792589686685526
R2:  0.033785520745597375


In [285]:
# Polynomial Regression
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(3)
X3_train = poly.fit_transform(X_train)
X3_test = poly.fit_transform(X_test)
runGridSearch(LinearRegression(), param_grid_lr, X3_train, y_train, X3_test, y_test)


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  5.7min remaining:  3.8min


KeyboardInterrupt: 

## Print Results of all Models

In [363]:
grid_results_df

,Name,MSE,MAE,Variance Score,R2
0,DecisionTreeRegressor,0.175964,0.291686,0.021140,0.021140
1,RandomForestRegressor,0.173300,0.290144,0.035956,0.035956
2,XGBRegressor,0.173153,0.285298,0.040201,0.036777


In [402]:
from xgboost import plot_tree
from matplotlib import pyplot as plt
fig, axes = plt.subplots(nrows = 1,ncols = 5,figsize = (10,2), dpi=3000)
for index in range(0, 5):
    plot_tree(xg_best_model.best_estimator_, 
              num_trees=index,              
              ax = axes[index]);
    
    axes[index].set_title('Estimator: ' + str(index), fontsize = 11)


In [366]:
list(pruned_data)

['crs_dep_time',
 'crs_arr_time',
 'crs_elapsed_time',
 'flights',
 'distance',
 'flight_speed',
 'flight_haul_type',
 'flight_month',
 'arrival_hour_of_day',
 'departure_hour_of_day',
 'percent_delay',
 'percent_flights',
 'avg_flights',
 'avg_passengers',
 'total_distance',
 'total_passengers',
 'avg_fuel',
 'avgCloud',
 'avgDryDays',
 'avgRainDays',
 'avgSnowDays',
 'avgThunderDays',
 'early_count',
 'early_avgDryDays',
 'early_avgRainDays',
 'early_avgCloud',
 'early_avgSnowDays',
 'delay_count',
 'delay_avgDryDays',
 'delay_avgRainDays',
 'delay_avgCloud',
 'delay_avgSnowDays']

In [386]:

from sklearn.inspection import permutation_importance
r = permutation_importance(xg_best_model.best_estimator_, 
                            X_test, y_test,
                            n_repeats=10,
                            random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{pruned_data.columns[i]:<8} => "
              f"{r.importances_mean[i]:.5f}"
              f" +/- {r.importances_std[i]:.5f}")

percent_delay => 0.01355 +/- 0.00175
crs_dep_time => 0.01134 +/- 0.00137
flight_speed => 0.00521 +/- 0.00073
crs_arr_time => 0.00509 +/- 0.00054
departure_hour_of_day => 0.00451 +/- 0.00085
crs_elapsed_time => 0.00370 +/- 0.00060
distance => 0.00326 +/- 0.00050
avg_fuel => 0.00281 +/- 0.00055
delay_count => 0.00265 +/- 0.00066
delay_avgDryDays => 0.00255 +/- 0.00055
arrival_hour_of_day => 0.00137 +/- 0.00040
avgRainDays => 0.00131 +/- 0.00054
avg_passengers => 0.00122 +/- 0.00035
early_avgDryDays => 0.00098 +/- 0.00019
avgDryDays => 0.00087 +/- 0.00040
avgCloud => 0.00059 +/- 0.00019
avgThunderDays => 0.00056 +/- 0.00023
avgSnowDays => 0.00047 +/- 0.00008


## Save the Model

In [437]:
import pickle

In [438]:
filename = 'xgb_best_model'
outfile = open(filename,'wb')
pickle.dump(xg_best_model,outfile)
outfile.close()

In [367]:
y_pred = xg_best_model.best_estimator_.predict(X_test)

In [372]:
print( np.unique( y_pred ) )
print("R2: ", metrics.r2_score(y_test, y_pred))
print("MAE: ", metrics.mean_absolute_error(y_test, y_pred))

[3.6769946 3.7817318 3.7891905 ... 4.35783   4.366303  4.6496587]
R2:  0.036776606593873495
MAE:  0.28529843585756504
